In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime

In [2]:
#获取202207往后12个月的月份list（为了后续读取文件）
month_date_list = list(pd.period_range('2022-07',periods=12,freq='M'))
monthlist = list(map(lambda x : str(x)[:4] + str(x)[-2:] , month_date_list))

In [3]:
#读取全部热搜文件并合并
monthallnewslist = []
for month in tqdm(monthlist):
    news = pd.read_excel(month + '.xlsx')
    monthallnewslist.append(news.drop('序号',axis=1))
newsall = pd.concat(monthallnewslist)
newsall.sort_values('上榜时间',inplace=True)
newsalldf = newsall.set_index('上榜时间')
newsalldf.index = pd.to_datetime(newsalldf.index)

100%|██████████| 12/12 [00:10<00:00,  1.10it/s]


In [4]:
#删去广告
newsalldf = newsalldf[newsalldf['备注']!='广告位']

In [5]:
#获取交易日
close = pd.read_feather('BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
tradeday = close['2022-07-01':].index

In [7]:
#构造交易日开盘时间，交易日停盘时间，前一个交易日开盘时间
start_tradeday = list(map(lambda x : x + datetime.timedelta(hours=9,minutes=15) , tradeday))
end_tradeday = list(map(lambda x : x + datetime.timedelta(hours=15) , tradeday))
previous_end_tradeday = end_tradeday.copy()
previous_end_tradeday.insert(0,pd.Timestamp('2022-06-30 15:00:00'))
previous_end_tradeday.pop()

Timestamp('2023-06-30 15:00:00')

In [10]:
#盘前news；盘中news
preopen_timepair_list = list(zip(previous_end_tradeday,start_tradeday))
opening_timepair_list = list(zip(start_tradeday,end_tradeday))

In [13]:
#获取三组热搜数据表
def getnewscut(newsdf,timepairlist1,timepairlist2):
    daynewslist = []
    timepairlist_pair = list(zip(timepairlist1,timepairlist2))
    for timepairs in timepairlist_pair:
        preopen_news = newsdf.truncate(before=timepairs[0][0], after=timepairs[0][1])
        opening_news = newsdf.truncate(before=timepairs[1][0], after=timepairs[1][1])
        day = timepairs[0][1].date()
        preopen_news = pd.concat([preopen_news,pd.Series(day,name='交易日',index=preopen_news.index),pd.Series('preopen',name='时间',index=preopen_news.index)],axis=1)
        opening_news = pd.concat([opening_news,pd.Series(day,name='交易日',index=opening_news.index),pd.Series('opening',name='时间',index=opening_news.index)],axis=1)
        daynewslist.append(preopen_news)
        daynewslist.append(opening_news)
    finalnews = pd.concat(daynewslist)
    return finalnews

In [14]:
all_news = getnewscut(newsalldf,preopen_timepair_list,opening_timepair_list)

In [21]:
all_news['热搜'] = all_news['热搜'].astype(str)
all_news.reset_index().to_feather('all_news.txt')